In [1]:
# scVI environment

import anndata as ad
import matplotlib.pyplot as plt
import mudata as md
import scanpy as sc
import numpy as np
import scvi
import seaborn as sns
import torch
import gc
import os
import sys
import pandas as pd

# Set global settings
scvi.settings.seed = 0
sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")

print("Last run with scvi-tools version:", scvi.__version__)

/home/users/kzlin/miniconda3/envs/scVI/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 0


Last run with scvi-tools version: 1.1.3


In [2]:
adata = ad.read_h5ad("/home/users/kzlin/kzlinlab/projects/subject-de/out/kevin/Writeup6/Writeup6_prater_scvi2_anndata.h5ad")

adata.obs["SeqBatch"] = adata.obs["SeqBatch"].astype('category')
adata.obs["Sex"] = adata.obs["Sex"].astype('category')
adata.obs["Race"] = adata.obs["Race"].astype('category')
adata.obs["genotype_APOE"] = adata.obs["genotype_APOE"].astype('category')
adata.obs["CERAD"] = adata.obs["CERAD"].astype('category')

# adding raw counts for referring to it in the future
adata.layers["counts"] = adata.X.copy()

# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)

sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    batch_key="Pt_ID",
    subset=True
)

pmi_replacement_values = {
    'D:1': 5.08,
    'D:10': 3.33,
    'D:3': 5.08,
    'D:15': 6.97
}

# Replace the NA values in the PMI column with the specified values
for pt_id, new_pmi in pmi_replacement_values.items():
    adata.obs.loc[adata.obs['Pt_ID'] == pt_id, 'PMI'] = new_pmi


/home/users/kzlin/miniconda3/envs/scVI/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]
/home/users/kzlin/miniconda3/envs/scVI/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]
/home/users/kzlin/miniconda3/envs/scVI/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated an

In [3]:
# https://docs.scvi-tools.org/en/stable/tutorials/notebooks/quick_start/api_overview.html
model = scvi.model.SCVI.load("/home/users/kzlin/kzlinlab/projects/subject-de/out/kevin/Writeup6/Writeup6_prater_scvi2-model", 
                             adata)

INFO     File                                                                                                      
         /home/users/kzlin/kzlinlab/projects/subject-de/out/kevin/Writeup6/Writeup6_prater_scvi2-model/model.pt    
         already downloaded                                                                                        


/home/users/kzlin/miniconda3/envs/scVI/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/users/kzlin/miniconda3/envs/scVI/lib/python3.9 ...
/home/users/kzlin/miniconda3/envs/scVI/lib/python3.9/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [4]:
print(model._adata.obs["Sex"].value_counts())
print(model._adata.obs["genotype_APOE"].value_counts())

Sex
F    91477
M    35890
Name: count, dtype: int64
genotype_APOE
3,3    75018
3,4    35753
4,4     9585
2,3     7011
Name: count, dtype: int64


In [25]:
# Extract the continuous and categorical covariates from the first 10 cells
continuous_covariates = ["percent.mito", "coded_Age", "nCount_RNA", "FreshBrainWeight", "PMI"]
categorical_covariates = ["Sex", "Race", "genotype_APOE", "CERAD", "SeqBatch"]

# Prepare the continuous covariates tensor
extra_continuous_covs_np = adata.obs[continuous_covariates].values
extra_continuous_covs_tensor = torch.from_numpy(extra_continuous_covs_np).float()

# Prepare the categorical covariates tensors
sex_tensor = torch.tensor(adata.obs['Sex'].astype('category').cat.codes.values, dtype=torch.long).squeeze()
race_tensor = torch.tensor(adata.obs['Race'].astype('category').cat.codes.values, dtype=torch.long).squeeze()
genotype_apoe_tensor = torch.tensor(adata.obs['genotype_APOE'].astype('category').cat.codes.values, dtype=torch.long).squeeze()
cerad_tensor = torch.tensor(adata.obs['CERAD'].astype('category').cat.codes.values, dtype=torch.long).squeeze()
seqbatch_tensor = torch.tensor(adata.obs['SeqBatch'].astype('category').cat.codes.values, dtype=torch.long).squeeze()



In [26]:

# Extract the latent representation for the first 10 cells
latent_representation = model.get_latent_representation(adata)
latent_representation_tensor = torch.tensor(latent_representation).float()


In [27]:
# Ensure all tensors are on the same device (e.g., CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
latent_representation_tensor = latent_representation_tensor.to(device)
extra_continuous_covs_tensor = extra_continuous_covs_tensor.to(device)
sex_tensor = sex_tensor.to(device)
race_tensor = race_tensor.to(device)
genotype_apoe_tensor = genotype_apoe_tensor.to(device)
cerad_tensor = cerad_tensor.to(device)
seqbatch_tensor = seqbatch_tensor.to(device)

In [28]:
# Concatenate the latent representation with the continuous covariates
decoder_input = torch.cat([latent_representation_tensor, extra_continuous_covs_tensor], dim=1)

In [31]:
# Print the expected number of categories for each categorical input
print("Expected number of categories for each categorical input:")
for idx, n_cat in enumerate(model.module.decoder.px_decoder.n_cat_list):
    print(f"Categorical input {idx + 1}: {n_cat}")

# Print the shape of the categorical tensors being passed
print(f"Shape of seqbatch_tensor: {seqbatch_tensor.shape}")
print(f"Shape of sex_tensor: {sex_tensor.shape}")
print(f"Shape of race_tensor: {race_tensor.shape}")
print(f"Shape of genotype_apoe_tensor: {genotype_apoe_tensor.shape}")
print(f"Shape of cerad_tensor: {cerad_tensor.shape}")

# Check unique values for each categorical tensor
print(f"Unique values in seqbatch_tensor: {seqbatch_tensor.unique()}")
print(f"Unique values in sex_tensor: {sex_tensor.unique()}")
print(f"Unique values in race_tensor: {race_tensor.unique()}")
print(f"Unique values in genotype_apoe_tensor: {genotype_apoe_tensor.unique()}")
print(f"Unique values in cerad_tensor: {cerad_tensor.unique()}")


Expected number of categories for each categorical input:
Categorical input 1: 2
Categorical input 2: 2
Categorical input 3: 4
Categorical input 4: 4
Categorical input 5: 4
Shape of seqbatch_tensor: torch.Size([127367])
Shape of sex_tensor: torch.Size([127367])
Shape of race_tensor: torch.Size([127367])
Shape of genotype_apoe_tensor: torch.Size([127367])
Shape of cerad_tensor: torch.Size([127367])
Unique values in seqbatch_tensor: tensor([0, 1])
Unique values in sex_tensor: tensor([0, 1])
Unique values in race_tensor: tensor([0, 1, 2, 3])
Unique values in genotype_apoe_tensor: tensor([0, 1, 2, 3])
Unique values in cerad_tensor: tensor([0, 1, 2, 3])


In [29]:
# Print the expected number of categories for each categorical input
print("Expected number of categories for each categorical input:")
for idx, n_cat in enumerate(model.module.decoder.px_decoder.n_cat_list):
    print(f"Categorical input {idx + 1}: {n_cat}")

# Check unique values for each categorical tensor
print(f"Unique values in seqbatch_tensor: {seqbatch_tensor.unique()}")
print(f"Unique values in sex_tensor: {sex_tensor.unique()}")
print(f"Unique values in race_tensor: {race_tensor.unique()}")
print(f"Unique values in genotype_apoe_tensor: {genotype_apoe_tensor.unique()}")
print(f"Unique values in cerad_tensor: {cerad_tensor.unique()}")


Expected number of categories for each categorical input:
Categorical input 1: 2
Categorical input 2: 2
Categorical input 3: 4
Categorical input 4: 4
Categorical input 5: 4
Unique values in seqbatch_tensor: tensor([0, 1])
Unique values in sex_tensor: tensor([0, 1])
Unique values in race_tensor: tensor([0, 1, 2, 3])
Unique values in genotype_apoe_tensor: tensor([0, 1, 2, 3])
Unique values in cerad_tensor: tensor([0, 1, 2, 3])


In [34]:
import torch.nn.functional as F

# One-hot encode the categorical covariates
sex_tensor_one_hot = F.one_hot(sex_tensor, num_classes=2).float()
race_tensor_one_hot = F.one_hot(race_tensor, num_classes=4).float()
genotype_apoe_tensor_one_hot = F.one_hot(genotype_apoe_tensor, num_classes=4).float()
cerad_tensor_one_hot = F.one_hot(cerad_tensor, num_classes=4).float()
seqbatch_tensor_one_hot = F.one_hot(seqbatch_tensor, num_classes=2).float()

In [35]:
seqbatch_tensor_one_hot

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])

In [36]:
seqbatch_tensor_one_hot.size(1)

2

In [37]:
predicted_gene_expression = model.module.decoder.px_decoder(
    decoder_input, 
    seqbatch_tensor_one_hot,  # Note: SeqBatch should be passed first if it was the first categorical covariate specified
    sex_tensor_one_hot, 
    race_tensor_one_hot, 
    genotype_apoe_tensor_one_hot, 
    cerad_tensor_one_hot
)

In [ ]:
genotype_apoe_tensor

In [ ]:
import inspect

# Print the signature of the forward method of px_decoder
print(inspect.signature(model.module.decoder.px_decoder.forward))

# # Or, you can directly inspect the entire method if it's not too large
# print(inspect.getsource(model.module.decoder.px_decoder.forward))

Preparing to decode

In [ ]:
type(model)

In [ ]:
model_attributes = model.__dict__
for key, value in model_attributes.items():
    print(f"{key}: {value}")

Set all the covariates to be the ones I want

In [ ]:
adata

In [ ]:
categorical_covariates = ["Sex", "Race", "SeqBatch", "genotype_APOE", "CERAD"]
continuous_covariates = ["percent.mito", "coded_Age", "nCount_RNA", "FreshBrainWeight", "PMI"]

In [ ]:
# following https://discourse.scverse.org/t/using-categorical-covariate-keys-when-sampling-or-generating-normalised-expression/1493
# manually set the mode and mean
# Store the original categories
original_categories = {covariate: adata.obs[covariate].cat.categories for covariate in categorical_covariates}

# Calculate mode for categorical covariates
for covariate in categorical_covariates:
    mode_value = adata.obs[covariate].mode()[0]
    adata.obs[covariate] = mode_value
    # Ensure the categorical variable type and categories are retained
    adata.obs[covariate] = adata.obs[covariate].astype('category')
    adata.obs[covariate] = adata.obs[covariate].cat.set_categories(original_categories[covariate])

# Calculate mean for continuous covariates
for covariate in continuous_covariates:
    mean_value = adata.obs[covariate].mean()
    adata.obs[covariate] = mean_value


In [ ]:
print(adata.obs['Sex'].value_counts())
print(model._adata.obs["Sex"].value_counts())

In [ ]:
# Let's just try for 10 cells
latent_representation = adata.obsm["X_scVI"][0:10,]
latent_representation

In [ ]:
extra_categorical_covs = adata.obs[categorical_covariates].iloc[:10]
extra_continuous_covs = adata.obs[continuous_covariates].iloc[:10]
print(extra_categorical_covs)
print(extra_continuous_covs)

In [ ]:
print(latent_representation.shape)
print(extra_continuous_covs.shape)
print(extra_categorical_covs.shape)

In [ ]:
type(extra_categorical_covs)

In [ ]:
# Convert DataFrames to NumPy arrays
extra_continuous_covs_np = extra_continuous_covs.values

# Encode categorical variables using one-hot encoding
extra_categorical_covs_encoded = pd.get_dummies(extra_categorical_covs, columns=extra_categorical_covs.columns)

# Convert the encoded DataFrame to a NumPy array
extra_categorical_covs_np = extra_categorical_covs_encoded.values

In [ ]:
# Convert the NumPy arrays to PyTorch tensors
extra_continuous_covs_tensor = torch.from_numpy(extra_continuous_covs_np).float()
extra_categorical_covs_tensor = torch.from_numpy(extra_categorical_covs_np).float()

# Concatenate the latent representation with the covariates
decoder_input = torch.cat([torch.from_numpy(latent_representation), 
                           extra_continuous_covs_tensor], dim=1)


In [ ]:
decoder_input

In [ ]:
decoder_input.shape

In [ ]:
sex_tensor = torch.tensor(adata.obs['Sex'].astype('category').cat.codes.values[:10]).to(extra_continuous_covs_tensor.device)
race_tensor = torch.tensor(adata.obs['Race'].astype('category').cat.codes.values[:10]).to(extra_continuous_covs_tensor.device)
genotype_apoe_tensor = torch.tensor(adata.obs['genotype_APOE'].astype('category').cat.codes.values[:10]).to(extra_continuous_covs_tensor.device)
cerad_tensor = torch.tensor(adata.obs['CERAD'].astype('category').cat.codes.values[:10]).to(extra_continuous_covs_tensor.device)

In [ ]:
# Print the dimensions to verify
print(f"sex_tensor shape: {sex_tensor.shape}")
print(f"race_tensor shape: {race_tensor.shape}")
print(f"genotype_apoe_tensor shape: {genotype_apoe_tensor.shape}")
print(f"cerad_tensor shape: {cerad_tensor.shape}")

In [ ]:
sex_tensor.shape

In [ ]:
# now get the seqBatch
SeqBatch_tensor = torch.tensor(adata.obs['SeqBatch'].astype('category').cat.codes.values[:10])
SeqBatch_tensor

In [ ]:
decoder_input.shape

In [ ]:
predicted_gene_expression = model.module.decoder.px_decoder(decoder_input, SeqBatch_tensor, sex_tensor, race_tensor, genotype_apoe_tensor, cerad_tensor)

In [ ]:
model.module.decoder.px_decoder.n_cat_list # uh oh....

In [ ]:
px_scale, px_r, px_rate, px_dropout = self.decoder(
    self.dispersion,
    decoder_input,
    size_factor,
    batch_index,
    *categorical_input,
    y)
